# ipa_utils

> Utilities for implementing Iterative Pairing Algorithm (IPA) routines using DiffPaSS.

In [ ]:
#| default_exp ipa_utils

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export

from collections import defaultdict

from diffpass.base import DiffPaSSResults

# Type aliases
BootstrapList = list  # List indexed by bootstrap iteration
GradientDescentList = list  # List indexed by gradient descent iteration
GroupByGroupList = list  # List indexed by group index

IndexPair = tuple[int, int]  # Pair of indices
IndexPairsInGroup = list[IndexPair]  # Pairs of indices in a group of sequences
IndexPairsInGroups = list[IndexPairsInGroup]  # Pairs of indices in groups of sequences

## Type aliases
```python
BootstrapList = list  # List indexed by bootstrap iteration
GradientDescentList = list  # List indexed by gradient descent iteration
GroupByGroupList = list  # List indexed by group index

IndexPair = tuple[int, int]  # Pair of indices
IndexPairsInGroup = list[IndexPair]  # Pairs of indices in a group of sequences
IndexPairsInGroups = list[IndexPairsInGroup]  # Pairs of indices in groups of sequences
```

In [ ]:
#| export

def get_robust_pairs(
    bootstrap_results: DiffPaSSResults,  # E.g. results of a run of `DiffPaSSModel.fit_bootstrap`
    cutoff: float = 1.0,  # Fraction of iterations a pair must be present in to be considered robust
) -> IndexPairsInGroups:  # Robust pairs of indices in each group of sequences
    """Get robust pairs of indices from a `DiffPaSSResults` object."""
    all_hard_perms = bootstrap_results.hard_perms
    group_sizes = [len(hp) for hp in all_hard_perms[0][0]]
    run_length = len(all_hard_perms)
    absolute_cutoff = cutoff * run_length

    all_pairings = defaultdict(int)
    for hard_perms_this_iter in all_hard_perms:
        for group_idx in range(len(group_sizes)):
            for idx, mapped_idx in enumerate(hard_perms_this_iter[-1][group_idx]):
                all_pairings[(group_idx, mapped_idx, idx)] += 1

    sorted_keys = sorted(all_pairings, key=lambda x: all_pairings[x], reverse=True)
    all_pairings_sorted = {k: all_pairings[k] for k in sorted_keys}

    idx_robust_pairs = 0
    for v in all_pairings_sorted.values():
        if v >= absolute_cutoff:
            idx_robust_pairs += 1
        else:
            break

    robust_pairs_by_group = [[] for _ in range(len(group_sizes))]

    for group_and_pairing_idxs in list(all_pairings_sorted)[:idx_robust_pairs]:
        group_idx, mapped_idx, idx = group_and_pairing_idxs
        robust_pairs_by_group[group_idx].append((mapped_idx, idx))

    return robust_pairs_by_group

In [ ]:
show_doc(get_robust_pairs)

---

[source](https://github.com/Bitbol-Lab/DiffPaSS/blob/main/diffpass/ipa_utils.py#L22){target="_blank" style="float:right; font-size:smaller"}

### get_robust_pairs

>      get_robust_pairs (bootstrap_results:diffpass.base.DiffPaSSResults,
>                        cutoff:float=1.0)

*Get robust pairs of indices from a `DiffPaSSResults` object.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| bootstrap_results | DiffPaSSResults |  | E.g. results of a run of `DiffPaSSModel.fit_bootstrap` |
| cutoff | float | 1.0 | Fraction of iterations a pair must be present in to be considered robust |
| **Returns** | **list** |  | **Robust pairs of indices in each group of sequences** |